# 第16章 string类和标准模板库

## 16.2 智能指针模板类

智能指针是行为类似于指针的类对象

以下代码中，每次调用时，在堆中分配的内存从来不回收，从而导致内存泄露。解决方法：在`return`之前添加 `delete`，来释放分配的内存。
```c
void remodel(std::string & str)
{
    std::string * ps = new std::string(str);
    ...
    str = ps;
    return;
}
```

如果其中有异常抛出，delete将不被执行，因此也会导致内存泄露。
```c
void remodel(std::string & str)
{
    std::string * ps = new std::string(str);
    ...
    if (weird_thing())
        throw exception();
    str = *ps;
    delete ps;
    return;
}
```

要是`ps`有一个析构函数，该析构函数在`ps`过期时释放它指向的内存。因此，`ps`的问题在于，它只是一个常规指针，不是有析构函数的类对象。如果它是对象，则可以在对象过期时，让它的析构函数删除指向的内存。这就是 `auto_ptr, unique_ptr, shared_ptr`背后的思想。模板`auto_ptr`是C++98提供的解决方案，C++11已经将其摒弃，并且提供了两种解决方案。

### 16.2.1 使用智能指针
这三个智能指针模板(`auto_ptr, unique_ptr, shared_ptr`)都定义了类似指针的对象，可以将`new`获得的地址赋给这种对象。当智能指针过期时，其析构函数将使用`delete`来释放内存。

要创建智能指针，必须包含头文件`memory`，该文件模板定义。模板`auto_ptr`包含如下构造函数
```c
template<class X>
class auto_ptr {
public:
    explicit auto_ptr(X* p=0) throw();
}
```

`explicit`的作用是用来声明类构造函数是显示调用的，而非隐式调用，所以只用于修饰单参构造函数。因为无参构造函数和多参构造函数本身就是显示调用的。再加上explicit关键字也没有什么意义。[C++ explicit 关键字](https://www.jianshu.com/p/af8034ec0e7a)

`throw()`意味着构造函数不会引发异常：与`auto_ptr`一样，`throw()`也被摒弃。

```c
auto_ptr<double> pd(new double);
auto_ptr<string> ps(new string);
```

下面是使用`auto_ptr`修改后的结果：
```c
#include <memory>
void remodel(std::string & str)
{
    std::auto_ptr<std::string> ps (new std::string(str));
    if (weird_thing())
        throw exception();
    str = *ps;
    // delete ps; NO LONGER NEEDED
    return;
}
```

所有智能指针类都有一个`explicit`构造函数，该构造函数将指针作为参数。因此不需要自动将指针转换为智能指针类对象。
```
shared_ptr<double> pd;
double *p_reg = new double;
pd = p_reg;                         // not allowed (implicit conversion)
pd = shared_ptr<double>(p_reg);     // allowed (explicit conversion)
shared_ptr<double> pshared = p_reg; // not allowed (implicit conversion)
shared_ptr<double> pshared(p_reg);  // allowed (explicit conversion)
```

三种指针都应该避免的一点：
pvac过期时，程序将把delete运算符用于非堆内存，这是错误的。
```c
string vacation("I wandered lonely as a cloud");
shared_ptr<string> pvac(&vacation);  // NO
```


### 16.2.2 有关智能指针的注意事项

```c
auto_ptr<string> ps (new string("I reingned lonely as a cloud."));
auto_ptr<string> vocation;
vocation = ps;
```

上述赋值语句将两个指针指向同一个`string`对象，这是不能接受的，因为程序将试图删除同一个对象两次--一次是`ps`过期时，一次时`vocation`过期时。要避免这种情况，方法有多种：
- 定义赋值运算符，使之执行深拷贝。这样两个指针将指向不同的对象，其中一个对象是另一个对象的副本。
- 建立所有权(ownership)的概念，对于特定的对象，只能有一个智能指针可以拥有它，这样只有拥有对象的智能指针的构造函数会删除该对象，然后，将赋值操作转让所有权。这就是用于`auto_ptr`和`unique_ptr`的策略，但`unique_ptr`的策略更严格。
- 创建智能更高的指针，跟踪引用特定对象的智能指针数。这称为引用计数(reference counting)。例如，赋值时，计数加一，指针过期时，计数减一。仅当最后一个指针过期时，才调用delete。这就是`shared_ptr`采用的策略。

### 16.2.3 unique_ptr为何优于auto_ptr
```c
unique_ptr<string> p3(new string("auto")); // #4
unique_ptr<string> p4;                     // #5
p4 = p3;                                   // #6
```
使用`unique_ptr`在编译阶段就会认为#6非法，因此`unique_ptr`比`auto_ptr`更安全。

相比于`auto_ptr`,`unique_ptr`还有另一个优点。它有一个可用于数组的变体。`auto_ptr`使用`delete`而不是`delete[]`,因此只能与`new`一起使用，而不能与`new[]`一起使用。但`unique_ptr`有使用`new[]`和`delete[]`的版本。
```c
std::unique_ptr<double []> pda(new double(5)); // will use delete []
```

### 16.2.4 选择智能指针
- 如果程序要使用多个指向同一个对象的指针时，应选择`shared_ptr`
- 如果程序不需要多个指向同一个对象的指针时，则可以使用`unique_ptr`。如果函数使用`new`分配内存，并返回指向该内存的指针，将其返回类型声明为`unique_ptr`是不错的选择。
